In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# data

In [3]:
DATASET = 'imagenet'
class_num   = 2
image_shape = (224, 224, 3)

batch_size = 32
train_size = None
test_size = 100

epochs = 50
eps = 0.03

In [4]:
def one_hot(x, k, dtype=np.float64):
    """Create a one-hot encoding of x of size k."""
    return onp.array(x[:, None] == onp.arange(k), dtype)

In [5]:
x_train = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_x_train.npy')[:2000]
y_train = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_y_train.npy')[:2000]

x_valid = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_x_train.npy')[2000:]
y_valid = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_y_train.npy')[2000:]

x_test = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_x_test.npy')
y_test = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_y_test.npy')

In [6]:
# shuffle
seed = 0
x_train, y_train = shaffle(x_train, y_train, seed)

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    5000
).batch(
    batch_size
).prefetch(10)

In [8]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
valid_ds = valid_ds.batch(20).prefetch(10)

# model

In [9]:
layers = tf.keras.layers

In [10]:
class BasicBlock(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.downsample = tf.keras.Sequential()
            self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                       kernel_size=(1, 1),
                                                       strides=stride))
            self.downsample.add(tf.keras.layers.BatchNormalization())
        else:
            self.downsample = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

def make_basic_block_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block

In [11]:
class ResNetTypeI(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=(2, 2),
                                            padding="same")
        
        self.maxpool = tf.keras.layers.MaxPool2D(strides=(2, 2))
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.layer1 = make_basic_block_layer(filter_num=64,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer(filter_num=128,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer(filter_num=256,
                                             blocks=layer_params[2],
                                             stride=2)
        self.layer4 = make_basic_block_layer(filter_num=512,
                                             blocks=layer_params[3],
                                             stride=2)
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(class_num)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.maxpool(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output

In [12]:
model = ResNetTypeI([2, 2, 2, 2])

In [13]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    else:
        return 1e-4

In [14]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.5),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [16]:
model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

Epoch 1/50
63/63 [==============================] - 15s 235ms/step - loss: 0.3145 - accuracy: 0.8745 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
63/63 [==============================] - 8s 122ms/step - loss: 0.1934 - accuracy: 0.9245 - val_loss: 0.4705 - val_accuracy: 1.0000
Epoch 3/50
63/63 [==============================] - 3s 43ms/step - loss: 0.1669 - accuracy: 0.9355 - val_loss: 0.3895 - val_accuracy: 0.9955
Epoch 4/50
63/63 [==============================] - 3s 43ms/step - loss: 0.1635 - accuracy: 0.9385 - val_loss: 0.2126 - val_accuracy: 1.0000
Epoch 5/50
63/63 [==============================] - 3s 40ms/step - loss: 0.1357 - accuracy: 0.9550 - val_loss: 0.2684 - val_accuracy: 0.9909
Epoch 6/50
63/63 [==============================] - 3s 40ms/step - loss: 0.1275 - accuracy: 0.9570 - val_loss: 0.1967 - val_accuracy: 0.9545
Epoch 7/50
63/63 [==============================] - 3s 40ms/step - loss: 0.0997 - accuracy: 0.9655 - val_loss: 0.2310 - val_accuracy: 0.9182
Ep

In [17]:
model.save_weights('./model_weights/resnet18_train=imagenet_all-without-DA_ce.h5')

In [18]:
model.evaluate(x=x_test, y=y_test, verbose=0)

[0.0686690377315972, 0.98]